In [1]:
!pip install transformers torch


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments


In [3]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

texts = [
    "User: I weigh 70 kg and prefer vegetarian meals.\nAI: Here’s a healthy vegetarian meal plan: Breakfast - Oatmeal with fruits, Lunch - Quinoa salad, Dinner - Lentil soup.",
    "User: I need a high-protein diet.\nAI: Try this: Breakfast - Greek yogurt with nuts, Lunch - Grilled chicken salad, Dinner - Baked salmon with veggies.",
    "User: I weigh 80 kg and want a low-carb diet.\nAI: Consider this: Breakfast - Avocado and eggs, Lunch - Chicken Caesar salad, Dinner - Steak with steamed broccoli."
]
encodings = tokenizer.batch_encode_plus(texts, padding=True, truncation=True, return_tensors='pt')

In [5]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none" # Disable wandb integration if you don't want to use it.
)


In [6]:
import wandb

In [7]:
wandb.login()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encodings['input_ids']
)
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sridharravi555 (sridharravi555-durham-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,past_key_values,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,encoder_hidden_states,encoder_attention_mask,labels,use_cache,output_attentions,output_hidden_states,return_dict,kwargs,label,label_ids,labels.

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

texts = [
    "User: I weigh 70 kg and prefer vegetarian meals.\nAI: Here’s a healthy vegetarian meal plan: Breakfast - Oatmeal with fruits, Lunch - Quinoa salad, Dinner - Lentil soup.",
    "User: I need a high-protein diet.\nAI: Try this: Breakfast - Greek yogurt with nuts, Lunch - Grilled chicken salad, Dinner - Baked salmon with veggies.",
    "User: I weigh 80 kg and want a low-carb diet.\nAI: Consider this: Breakfast - Avocado and eggs, Lunch - Chicken Caesar salad, Dinner - Steak with steamed broccoli."
]
encodings = tokenizer.batch_encode_plus(texts, padding=True, truncation=True, return_tensors='pt')

# Create a dataset from the encodings
# We assume the input_ids are also the labels for language modeling
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone() # for language modeling, labels are the same as input_ids
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = MyDataset(encodings)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none"  # Disable wandb integration if you don't want to use it.
)

import wandb
wandb.login()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset  # Use the custom dataset
)
trainer.train()

<ipython-input-8-71fd7caef044>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.64299201965332, metrics={'train_runtime': 30.2782, 'train_samples_per_second': 0.099, 'train_steps_per_second': 0.099, 'total_flos': 65833344000.0, 'train_loss': 4.64299201965332, 'epoch': 1.0})

In [10]:
input_text = "I weigh 70 kg and prefer vegetarian meals"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids, max_length=50, num_return_sequences=1)
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I weigh 70 kg and prefer vegetarian meals.

The study, which was published in the journal PLOS ONE, found that the vegetarian diet was associated with a lower risk of death from cardiovascular disease.

"The results of the study suggest
